In [5]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt

from cholesky import Cholesky
from imutils import img2array, array2img, load_coords, homogenize
from interpolation import affine_grid_generator, bilinear_sampler

plt.rcParams['figure.figsize'] = (9.0, 9.0)

In [6]:
switch = False

In [7]:
# load and homogenize coordinates 
x, y = load_coords('coords.p')

if switch:
    x = homogenize(x)
else:
    y = homogenize(y)

print("y: {}".format(y.shape))
print("x: {}".format(x.shape))

y: (9, 3)
x: (9, 2)


In [8]:
# create P array
num_pts = y.shape[0]

if switch:
    P = []
    for i in range(num_pts):
        P.append(np.hstack([x[i], np.zeros(3).T]))
        P.append(np.hstack([np.zeros(3).T, x[i]]))
    P = np.array(P)
    Q = y.flatten()

else:
    P = []
    for i in range(num_pts):
        P.append(np.hstack([y[i], np.zeros(3).T]))
        P.append(np.hstack([np.zeros(3).T, y[i]]))
    P = np.array(P)   
    Q = x.flatten()

print("P: {}".format(P.shape))
print("Q: {}".format(Q.shape))

P: (18, 6)
Q: (18,)


In [9]:
M = Cholesky(np.dot(P.T, P)).solve(np.dot(P.T, Q))
M = M.reshape(2, 3)
M[np.abs(M) < 1e-10] = 0

In [ ]:
M

In [ ]:
M[0, 2]=0
M[1, 2]=0

In [ ]:
# going from x to y

# zoom out
H1 = np.array([[1.5, 0], [0, 1.5]])

# rotate by -45
H2 = np.array([[0.707, -0.707], [0.707, 0.707]])

# combine
H = np.dot(H2, H1)

# make affine
zeros = np.zeros((H.shape[0], 1), dtype=H.dtype)
H = np.append(H, zeros, axis=1)
print(H)

In [ ]:
img = img2array('./imgs/crooked_cat.png', desired_size=(800, 800))

affine_grid = affine_grid_generator(800, 800, M)

x_s = affine_grid[0:1, :].squeeze()
y_s = affine_grid[1:2, :].squeeze()

out = bilinear_sampler(img, x_s, y_s)
out = np.clip(out, 0, 1)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2)

ax[0].imshow(img)
ax[0].get_xaxis().set_visible(False)
ax[0].get_yaxis().set_visible(False)
ax[0].set_title("Crooked")
ax[1].imshow(out)
ax[1].get_xaxis().set_visible(False)
ax[1].get_yaxis().set_visible(False)
ax[1].set_title("Fixed")

plt.show()

In [ ]:
def inv_trans(M):
    row = np.zeros((1, M.shape[1]), dtype=M.dtype)
    row[-1] = 1
    M = np.append(M, row, axis=0)
    return np.linalg.inv(M)[0:2, :]